# Metrics evaluation notebook

_Alex Malz (NYU)_
_Renee Hlozek (Toronto)_
_Rahul Biswas (Stockholm University)_
_Tarek Alam (UCL)_
_Rafael Martinez-Galarza (Harvard)_
_Anita Bahmanyar (Toronto)_


This notebook defines the metrics (and corresponding truth tables) to be run via an input file and to produce plots based on the output.

In [ ]:
import string
import itertools
import random
import numpy as np
import sklearn as skl
from sklearn import metrics
import pandas

import proclam
from proclam import *

In [ ]:
import matplotlib as mpl
# import matplotlib.pyplot as plt
mpl.use('Agg')
mpl.rcParams['text.usetex'] = False
mpl.rcParams['mathtext.rm'] = 'serif'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'Times New Roman'
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['savefig.dpi'] = 250
mpl.rcParams['savefig.format'] = 'pdf'
mpl.rcParams['savefig.bbox'] = 'tight'
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)
        
def per_metric_helper(ax, n, data, metric_names, codes, shapes, colors):
    plot_n = n+1
    in_x = np.arange(len(codes))
    ax_n = ax
#     ax.scatter(in_x, [-1]*len(codes), color='k', alpha=0, marker=shapes[n], s=50, label=metric_names[n])
    n_factor = 0.1 * (plot_n - 2)
    if plot_n>1:
        ax_n = ax.twinx()
        rot_ang = 270
        label_space = 15.
    else:
        rot_ang = 90
        label_space = 0.
    if plot_n>2:
        ax_n.spines["right"].set_position(("axes", 1. + 0.1 * (plot_n-1)))
        make_patch_spines_invisible(ax_n)
        ax_n.spines["right"].set_visible(True)
    
#     if n != 0:
#         data[n] *= -1.
#     ax_n.semilogy()
        
    print('trying to plot')
    print((in_x+n_factor*np.ones_like(data[n]), data[n]))
    handle = ax_n.scatter(in_x+n_factor*np.ones_like(data[n]), data[n], marker=shapes[plot_n], s=10, color=colors[n], label=metric_names[n])
    ax_n.set_ylabel(metric_names[n], rotation=rot_ang, fontsize=14, labelpad=label_space)
    print('plotted '+metric_names[n]+': '+str(data[n]))
    return(ax, ax_n, handle)

def metric_plot(codes, metric_names, data, shapes, colors):
    xs = np.arange(len(codes))
    
    fig, ax = plt.subplots()
    fig.subplots_adjust(right=1.)
    handles = []
#     for name in metric_names:
#         handles.append(ax.scatter([0], [0], alpha=0.))
    for n in range(len(metric_names)):
        (ax, ax_n, handle) = per_metric_helper(ax, n, data, metric_names, codes, shapes, colors)
        handles.append(handle)
#     plt.xticklabels(codes)
    plt.xticks(xs, codes)#, rotation='vertical')
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)
    plt.xlabel('Classifiers', fontsize=14)
    plt.legend(handles, metric_names)
    plt.savefig('metrics.png')
    return(fig)

## Define the inputs

In [ ]:
metricslist = ['Brier', 'LogLoss']
colors = ['b', 'r']
n_symb = 5
shapes = [(n_symb, 3, 0), (n_symb, 0, 0), (n_symb, 1, 0), (n_symb, 2, 0)]
dirname = 'proclam/classification_probabilities/'
names = ['RandomForest', 'KNeighbors', 'MLPNeuralNet']

classifications = ['%s/predicted_prob_%s.csv'%(name,name) for name in names] #, 'KNeighbors/predicted_prob_KNeighbors.csv', 'MLPNeuralNet/predicted_prob_MLPNeuralNet.csv']
truth_tables = ['%s/truth_table_%s.csv'%(name,name) for name in names] #, 'KNeighbors/truth_table_KNeighbors.csv', 'MLPNeuralNet/truth_table_MLPNeuralNet.csv']
print(np.shape(truth_tables))
class_pairs = zip(classifications, truth_tables)


In [ ]:
def read_class_pairs(pair):
    import pandas as pd
    clfile = pair[0]
    truthfile = pair[1]

    prob_mat = pd.read_csv(dirname+clfile, delim_whitespace=True).values
    nobj = np.shape(prob_mat)[0]
    nclass = np.shape(prob_mat)[1]
    print(nclass,nobj)
    truth_values = pd.read_csv(dirname+truthfile, delim_whitespace=True).values
    nobj_truth = np.shape(truth_values)[0]
    nclass_truth = np.shape(truth_values)[1]
    print(nclass_truth, nobj_truth)
    tvec = np.where(truth_values==1)[1]
    if nclass_truth!= nclass:
        print('Truth table of size %i x %i and prob matrix of size %i x %i do not match up in size'%(nobj,nclass,nobj_truth,nclass_truth))
    else:
        print('Considering classifications with %i classes'%nclass)
    pmat = prob_mat
    return pmat, tvec

In [ ]:
markerlist = ['s', '*', 'o']
# plt.figure(2, figsize=(10,8))
data = np.empty((len(metricslist), len(names)))
print('data shape'+str(np.shape(data)))
for cc, pair in enumerate(class_pairs):
    probm, truthv = read_class_pairs(pair)
    for count, metric in enumerate(metricslist):
        print('Using metric %s'%metric)
        D = getattr(proclam.metrics, metric)()
        hm = D.evaluate(probm,truthv)
        print(hm)
        print('indices'+str((cc, count)))
        data[count][cc] = hm
print(np.shape(data)) 
#         plt.plot(cc, hm, marker=markerlist[count])
        
#     plt.ylabel('Brier score')
#     plt.xlabel('Metrics')
#    ax = pl.gca()
#    ax.set_xticklabels(names)
    #print(cm, tm)
#from proclam.simulators import simulator
#from proclam.simulators import logunbalanced
metric_plot(names, metricslist, data, markerlist, colors)

In [ ]:
# make a plot of classifier on the x-axis and metric score on the y-axis
